In [103]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import pandas as pd
import pickle
import numpy as np

import sklearn
from sklearn.metrics import roc_auc_score, roc_curve


In [2]:
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

/home/dalab5/miniconda3/envs/GNN/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
X.shape

(150, 4)

In [21]:
y.shape

(150,)

# Load data
- fv
- peer
    - tnic similarity
    - past event
    
    
each firm could form multiple observations (as long as there is no missing values)


$$\hat{logit} = X^{\top} \beta$$

- logit : (N_samples, )
- X : (N_samples, dim)
    - X[i, :] = $[fv \; ; top\;10\;tnic\;avg ; Num\;of\;peer\;event_{t-1}]$

In [12]:
with open(data_path+f"/dataset_top10_freq5_1997_2019.pickle", "rb") as f:
    arr_cs, arr_bs, timelines = pickle.load(f)

In [18]:
with open(tmp_data_path+f'/a5_top_10_peers_tnic2_1997.pickle', 'rb') as f:
    tnic = pickle.load(f) 

In [16]:
sdc_tnic = pd.read_pickle(tmp_data_path+f"/sdc_tnic_1997_2020") # always load full dataset
sdc_tnic = same_day_only_one(sdc_tnic) 

# TNIC structure
with open(tmp_data_path+f"/tnic_info_3_pairs_{1997-1}_{2020-1}", 'rb') as f: # read full
    gvkey_lsts, key_ind_maps , ind_key_maps = pickle.load(f)

# frequent acquirer 
with open(data_path+f"/freq_a_info_1997_2020.pickle", "rb") as f: 
    A_freq, a_freq_lst, a_freq_idx_to_gvkey_mapping, a_freq_gvkey_to_idx_mapping = pickle.load(f)


shape before removing same-day multi events: (9661, 50)
shape after removing same-day multi events: (9448, 50)


In [17]:
sdc_tnic.columns

Index(['AGVKEY', 'DA', 'ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC',
       'AUPBL', 'AUPNAMES', 'AUPPUB', 'BLOCK', 'CREEP', 'DE', 'STATC', 'SYNOP',
       'VAL', 'PCTACQ', 'PSOUGHTOWN', 'PSOUGHT', 'PHDA', 'PCTOWN', 'PSOUGHTT',
       'PRIVATIZATION', 'DEAL_NO', 'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC',
       'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB', 'SIC_A', 'SIC_T',
       'YEAR', 'AS_PERMNO', 'AP_PERMNO', 'TS_PERMNO', 'TP_PERMNO', 'AS_GVKEY',
       'AP_GVKEY', 'TS_GVKEY', 'TP_GVKEY', 'GVKEY_STATUS', 'MA_TYPE',
       'TGVKEY'],
      dtype='object')

In [13]:
timelines[0]

,GLOBAL_IDX,LOCAL_IDX,UPDATE_DATE,EVENT_TYPE,TGVKEY
0,0,0,1997-01-01,3,NaN
1,1,0,1997-01-06,2,10499
2,2,1,1997-01-16,2,5073
3,3,2,1997-01-23,1,11636
4,4,3,1997-03-10,1,29132
...,...,...,...,...,...
212,212,191,2017-10-31,2,162894
213,213,21,2018-01-01,3,NaN
214,214,192,2018-02-07,1,135990
215,215,193,2018-06-06,1,14934


In [26]:
np.mean(tnic['5047'].score.values)

0.03302

# Creating pre-processed data

## get tnic_top_10_avg per year per gvkey



In [49]:
top_10_tnic_avg = {}
for year in range(1996, 2020):
    with open(tmp_data_path+f'/a5_top_10_peers_tnic2_{year}.pickle', 'rb') as f:
        tnic = pickle.load(f) 
    yearly = {}
    for gvkey in a_freq_lst:
        yearly[gvkey] = np.mean(tnic[gvkey].score.values)
    
    top_10_tnic_avg[year] = yearly    


In [50]:
top_10_tnic_avg[1998]['5047']

0.044533333333333334

## get peer MA historical event


use t-4, so must start from 1997 + 4 = 2001

In [31]:
timelines[0]

,GLOBAL_IDX,LOCAL_IDX,UPDATE_DATE,EVENT_TYPE,TGVKEY
0,0,0,1997-01-01,3,NaN
1,1,0,1997-01-06,2,10499
2,2,1,1997-01-16,2,5073
3,3,2,1997-01-23,1,11636
4,4,3,1997-03-10,1,29132
...,...,...,...,...,...
212,212,191,2017-10-31,2,162894
213,213,21,2018-01-01,3,NaN
214,214,192,2018-02-07,1,135990
215,215,193,2018-06-06,1,14934


In [37]:

peer_event_history = {}
for i, timeline in enumerate(timelines):
    focal_gvkey = a_freq_idx_to_gvkey_mapping[i]
    yearly = {}
    for year in range(2001, 2018):
        backs = {}
        for back in range(0, 5):
            sub_df = timeline[(timeline.UPDATE_DATE <= np.datetime64(f"{year-back}-12-31")) & (timeline.UPDATE_DATE >= np.datetime64(f"{year-back}-01-01")) & (timeline.EVENT_TYPE == '2')]
            n_event = sub_df.shape[0]
            backs[f"t-{back}"] = n_event
        yearly[year] = backs
    peer_event_history[focal_gvkey] = yearly
            
        
        
    

In [43]:
peer_event_history['5047'][2001]

{'t-0': 4, 't-1': 18, 't-2': 17, 't-3': 3, 't-4': 6}

## get fv

In [51]:
fvs = {}
for gvkey in a_freq_lst:
    i = a_freq_gvkey_to_idx_mapping[gvkey]
    timeline = timelines[i]
    arr_b = arr_bs[i]
    yearly = {}
    for year in range(2001, 2018):
        sub_df = timeline[(timeline.UPDATE_DATE == np.datetime64(f"{year+1}-01-01")) & (timeline.EVENT_TYPE == '3')]
        if sub_df.shape[0] != 0:
            idx_b = sub_df.iloc[-1, 1]
            fv = arr_b[idx_b]
        else:
            fv = 0 ####### !
        yearly[year] = fv
    
    fvs[gvkey] = yearly
        
            
        
        

In [54]:
fvs['5047'][2003]

array([8.14099012e-01, 1.67960618e-01, 1.59232474e-02, 2.12883048e-06,
       1.03654689e-06, 4.74701633e-07, 5.48090743e-07, 4.69021662e-07,
       5.63625855e-07, 7.03834577e-07, 5.28638157e-06, 6.91973231e-07,
       4.72330886e-07, 6.02324768e-07])

## get y

In [55]:
ys = {}
for gvkey in a_freq_lst:
    i = a_freq_gvkey_to_idx_mapping[gvkey]
    timeline = timelines[i]
    yearly = {}
    for year in range(2001+1, 2018+1): ######### !
        sub_df = timeline[(timeline.UPDATE_DATE <= np.datetime64(f"{year}-12-31")) & (timeline.UPDATE_DATE >= np.datetime64(f"{year}-01-01")) & (timeline.EVENT_TYPE == '1')]
        if sub_df.shape[0] != 0:
            yearly[year] = 1
        else:
            yearly[year] = 0
    
    ys[gvkey] = yearly
        
    

In [58]:
ys['5047'][2004]

1

# creating X, y

In [97]:
X = []
y = []
for gvkey in a_freq_lst:
    for year in range(2001, 2018):
        
        # X
        fv = fvs[gvkey][year] # 16
        
        lag = []
        for back in range(0, 5):
            lag.append(peer_event_history[gvkey][year][f't-{back}'])
        lag = np.array(lag)  # 5

        compe = np.array(top_10_tnic_avg[year][gvkey]).reshape(1) # 1

        
        if isinstance(fv, (np.ndarray, np.generic)) & (~np.isnan(compe)): ######## !       
            #x = np.concatenate((fv, lag, compe)) # 
            x = fv
            X.append(x)

            # y
            y.append(ys[gvkey][year+1])        

X = np.array(X)
y = np.array(y)

In [98]:
X.shape

(6987, 14)

In [99]:
y.shape

(6987,)

# run

In [101]:
clf = LogisticRegression(random_state=0).fit(X, y)
print(roc_auc_score(y, clf.predict_proba(X)[:, 1]))

0.5451156629716286
